In [1]:
import numpy as np
import cv2

from keras.models import load_model

import itertools

Using TensorFlow backend.


In [2]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.7
set_session(tf.Session(config=config))

In [ ]:
FRAME_IN_SAMPLE = 12
W_STRIDE = 4
H_STRIDE = 4
SIZE_STRIDE = 8

WIDTH = 80
HEIGHT = 45

MODEL_FILE = "model_v2.h5"

file_name = "data/fire_positive_prepared_test/03.mp4"

model = load_model(MODEL_FILE)

cap = cv2.VideoCapture(file_name)
stream_fps = cap.get(cv2.CAP_PROP_FPS)
stream_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
stream_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

all_y = np.arange(0, stream_height, H_STRIDE)
all_x = np.arange(0, stream_width, W_STRIDE)
all_rectangle = list(itertools.product(all_y, all_x, [WIDTH], [HEIGHT]))

current_sample = []
fullscreen_sample = np.zeros((1, FRAME_IN_SAMPLE, stream_height, stream_width, 3))
rectangles = []

while cap.isOpened():
    
    ret, frame = cap.read()
    if frame is None:
        break
    
    current_sample.append(frame)
    if len(current_sample) == FRAME_IN_SAMPLE:
        rectangles = []
        fullscreen_sample[0, :, :, :, :] = np.asarray(current_sample).astype(np.float32) / 255
        current_sample = []
        
        all_rectangle_filtered = filter(lambda c: (c[0] + c[3] < stream_height) and (c[1] + c[2] < stream_width), all_rectangle)
        for y, x, w, h in all_rectangle_filtered:
            sample = fullscreen_sample[:, :, y:y+h, x:x+w,:]
            pred = model.predict(x = [sample])
            
            #print(y)
            if pred[0][0] > 0.95:
                rectangles.append((y, x, w, h))
    
    #print(len(rectangles))
    for y, x, w, h in rectangles:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
    
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
            
cap.release()
cv2.destroyAllWindows()

In [4]:
cap.release()
cv2.destroyAllWindows()

In [6]:
#model.predict(sample)
#model.predict(x = sample)
#?model.predict
#sample
model.predict(x = [sample])[0][0]

0.90038097

In [18]:
all_y = np.arange(0, stream_height, H_STRIDE)
all_x = np.arange(0, stream_width, W_STRIDE)
all_rectangle = list(itertools.product(all_y, all_x, [WIDTH], [HEIGHT]))

all_rectangle_filtered = filter(lambda c: (c[0] + c[3] < stream_height) and (c[1] + c[2] < stream_width), all_rectangle)
len(list(all_rectangle_filtered))

11060